# Pendulum simulation and phase portrait

# To launch selected cell use `Ctrl + Enter`

# Requirements

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from ipywidgets import FloatSlider
from IPython.display import display
from manim import *
config.media_width = "75%"
config.verbosity = "WARNING"

Manim Community v0.17.3

# Setting the system parameters

# $\theta''+\frac{b}{m}\theta'+\frac{l}{g}\sin{\theta}=0$ is our equation

An explanation of all the maths will be here later. Now you can change parameters and see where it leads. I don't recommend setting high angles because the phase portrait will just go out of frame, it's not done yet.

In [2]:
Slider_angle = FloatSlider(
    value=np.e,
    min=-np.pi, 
    max=np.pi, 
    step=0.01, 
    description='Start angle:',
    continuous_update=False,
    orientation='horizontal'
)
display(Slider_angle)

Slider_friction = FloatSlider(
    value=0.35,
    min=0, 
    max=1, 
    step=0.01, 
    description='Friction:',
    continuous_update=False,
    orientation='horizontal'
)
display(Slider_friction)

Slider_m = FloatSlider(
    value=1,
    min=0, 
    max=100, 
    step=0.01, 
    description='Mass:',
    continuous_update=False,
    orientation='horizontal'
)
display(Slider_m)

Slider_g = FloatSlider(
    value=1,
    min=0, 
    max=100, 
    step=0.01, 
    description='Gravity:',
    continuous_update=False,
    orientation='horizontal'
)
display(Slider_g)

Slider_T = FloatSlider(
    value=30,
    min=0, 
    max=100, 
    step=0.01, 
    description='Time:',
    continuous_update=False,
    orientation='horizontal'
)
display(Slider_T)

FloatSlider(value=2.718281828459045, continuous_update=False, description='Start angle:', max=3.14159265358979…

FloatSlider(value=0.35, continuous_update=False, description='Friction:', max=1.0, step=0.01)

FloatSlider(value=1.0, continuous_update=False, description='Mass:', step=0.01)

FloatSlider(value=1.0, continuous_update=False, description='Gravity:', step=0.01)

FloatSlider(value=30.0, continuous_update=False, description='Time:', step=0.01)

# Solution calculation

In [3]:
b, m, g, l, T = Slider_friction.value, Slider_m.value, Slider_g.value, 1, Slider_T.value # T is simulation time

def dSdt(t,S):
    x1, x2 = S
    return [x2, -b/m*x2-g/l*np.sin(x1)]
S0 = (Slider_angle.value, 0) # initial conditions

t = np.linspace(0, T, 10001)
sol = solve_ivp(dSdt, t_span=(0,max(t)), y0=S0, t_eval=t) #, method = 'DOP853', rtol=1e-10, atol=1e-13)

time_range = tuple(sol.t)
y_angles = sol.y[0]
y_speeds = sol.y[1]

# Animation (it takes some time, be patient)

In [4]:
%%manim -qm Phase_Space

Pendulum_Point = 3.5*LEFT + 0.5*UP
Scale_Pendulum = 3
Phase_Point = 2.5*RIGHT
Scale_Phase = 4.5/3.2

def get_continuous_y(t, time_range, y_values):
    
    t_nearest = min(time_range, key=lambda x: abs(x-t))
    i = time_range.index(t_nearest)

    if t < t_nearest:
        i1, i2 = i-1, i
    elif t > t_nearest:
        i1, i2 = i, i+1
    else:
        return y_values[i]
        
    t1, t2, y1, y2 = time_range[i1], time_range[i2], y_values[i1], y_values[i2]

    return (t - t1) / (t2 - t1) * (y2 - y1) + y1
        
class Phase_Space(Scene):
    
    def phase(self, t):
        return Phase_Point + Scale_Phase*np.array((get_continuous_y(t, time_range, y_angles), get_continuous_y(t, time_range, y_speeds), 0))
    
    def pend(self, t):
        return Pendulum_Point + Scale_Pendulum*np.array((np.sin(get_continuous_y(t, time_range, y_angles)), -np.cos(get_continuous_y(t, time_range, y_angles)), 0))
    
    def construct(self):
        
        t = ValueTracker(0)
        
        ### Axes
        
        axes = Axes(
            x_range=[-3.2, 3.2],
            y_range=[-2.6, 2.6],
            x_length=6.4*Scale_Phase, 
            y_length=5.2*Scale_Phase,            
            axis_config={                
                "include_tip": True,                
                #"color": GREY,
                "stroke_width": 2,
                "font_size": 24, 
                "tick_size": 0.07,                
                "longer_tick_multiple": 1.5,
                "line_to_number_buff": 0.15,  
                "decimal_number_config": {
                    #"color": ORANGE,
                    "num_decimal_places": 0
                    },             
                },
            x_axis_config={
                "tip_width": 0.15,
                "tip_height": 0.15,
                "numbers_to_include": np.arange(-3, 4, 1),
                "numbers_with_elongated_ticks": [-1, 1],
                },             
            y_axis_config={
                "tip_width": 0.15,
                "tip_height": 0.15,                
                "numbers_to_include": np.arange(-2, 3, 1),
                "tick_size": 0.08,
                "font_size": 25, 
                "numbers_with_elongated_ticks": [-1, 1],                
                }
        ).move_to(Phase_Point)
        x_lab = axes.get_x_axis_label("\\theta", direction=UP, buff=0.2)
        y_lab = axes.get_y_axis_label("\\theta'", direction=RIGHT, buff=0.2)
        labels = VGroup(x_lab.scale(1), y_lab.scale(1))

        self.play(Write(axes, run_time=1), lag_ratio=0.2)
        
         ### Phase space
        
        point = always_redraw(lambda: Dot(self.phase(t.get_value()), radius=0.07).set_color(YELLOW))
        trace1 = TracedPath(point.get_center, dissipating_time=T, stroke_opacity=[1, 1], stroke_color=YELLOW, stroke_width=6, z_index=1)
        self.play(Write(labels))        
        
        ### Pendulum
        
        dashed_line = DashedLine(ORIGIN + Pendulum_Point, Scale_Pendulum*DOWN + Pendulum_Point).set_color(GRAY)
        mass = always_redraw(lambda: Dot(self.pend(t.get_value()), radius=0.3).set_color(BLUE))
        line = always_redraw(lambda: Line(ORIGIN + Pendulum_Point, self.pend(t.get_value())))
        trace2 = TracedPath(mass.get_center, dissipating_time=0.3, stroke_opacity=[0, 1], stroke_color='#FF8C00', stroke_width=10, z_index=1)
        
        ### Angle
        
        def theta(x):
            if x >= 0:
                return Angle(dashed_line, line, radius=1, quadrant=(1,1), stroke_width=8, other_angle=False, color=YELLOW)
            else:
                return Angle(dashed_line, line, radius=1, quadrant=(1,1), stroke_width=8, other_angle=True, color=YELLOW)
            
        def label_scale(x):
            if x > np.pi:
                while x > np.pi:
                    x = x - 2*np.pi
            elif x < -np.pi:
                while x < -np.pi:
                    x = x + 2*np.pi
                    
            x = abs(x)
            
            if x > 1:
                return 1
            else:
                return np.power(x, 1/3)
            
        
        angle_theta = always_redraw(lambda: theta(get_continuous_y(t.get_value(), time_range, y_angles)))
        label_theta = always_redraw(lambda: MathTex("\\theta", color=YELLOW, z_index=1).scale(1.5*label_scale(get_continuous_y(t.get_value(), time_range, y_angles))).next_to(angle_theta, DOWN))
        
        ###
        
        self.play(
            AnimationGroup(
                Create(dashed_line),
                Create(line),
                Create(angle_theta),
                Write(label_theta),
                DrawBorderThenFill(mass),
                FadeIn(point),
                lag_ratio=0.1,
                run_time=1
            )
        )
        self.add(trace1)
        self.add(trace2)
        
        self.wait()
        
        self.play(t.animate.set_value(T), run_time=T, rate_func=linear)

In the next version, the phase space will be represented as a cylinder, which will make it possible to draw some topological conclusions.